#10x RNA-seq gene expression data (part 2c)

We perform subsetting of the Whole Mouse Brain Atlas at a particular taxonomic level. This allows us to create manageable `h5ad` files of certain branches of the taxonomy. 

You need to be connected to the internet to run this notebook and have run through the [getting started notebook](https://alleninstitute.github.io/abc_atlas_access/notebooks/getting_started.html).

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import anndata
import time

from abc_atlas_access.abc_atlas_cache.abc_project_cache import AbcProjectCache

We will interact with the data using the **AbcProjectCache**. This cache object tracks which data has been downloaded and serves the path to the requsted data on disk. For metadata, the cache can also directly serve a up a Pandas Dataframe. See the getting_started notebook for more details on using the cache including installing it if it has not already been.

**Change the download_base variable to where you have downloaded the data in your system.**

In [ ]:
download_base = Path('../../data/abc_atlas')
abc_cache = AbcProjectCache.from_cache_dir(download_base)
abc_cache.list_manifest_file_names

Read in the expanded cell metadata table from the cache. Examples of creating this table are presented in part 1.

In [ ]:
# load cell metadata
cell_metadata = abc_cache.get_metadata_dataframe(directory='WMB-10X', file_name='cell_metadata_with_cluster_annotation')
cell_metadata.set_index('cell_label',inplace=True)
cell_metadata.head()

In [ ]:
# list columns of the metadata
list(cell_metadata)

In [ ]:
# generate a table of matrices
matrices = cell.groupby(['dataset_label', 'feature_matrix_label'])[['library_label']].count()
matrices.columns  = ['cell_count']
matrices

Below we define a function to extract whole transcriptomic data from selected cells across all matrices in the database. We supply a taxonomic level (`target_grouping_level`) to perform the subsetting on and a `target_grouping_label` which represents the name of the group that we hope to capture.

In [ ]:
def extract_grouping_to_h5ad(
    cell_metadata: pd.DataFrame,
    feature_matrices_info: pd.DataFrame,
    target_grouping_label: str,
    target_grouping_level: str,
    output_filename: str,
    data_directory: str = None, # Optional: if not using abc_atlas_cache directly
    limit_matrices: int = None, # Optional: for testing, limit number of matrices processed
    additional_metadata_cols: list = None # New: List of column names from cell_metadata to add to .obs
) -> str:
    """
    Extracts all cells belonging to a specified subclass from multiple feature matrices
    and saves them into a new AnnData (.h5ad) file, keeping all genes.
    Optionally merges additional metadata columns from the cell_metadata DataFrame
    into the .obs DataFrame of the resulting AnnData object.

    Args:
        cell_metadata (pd.DataFrame): DataFrame containing cell metadata,
            must include 'feature_matrix_label' and 'subclass_label' columns,
            with cell IDs as index.
        feature_matrices_info (pd.DataFrame): DataFrame containing information about
            feature matrices, with (dataset_id, matrix_path) as index.
        target_grouping_level (str): The level of the taxonomy that you want to select 
            from (e.g., 'subclass', 'supertype')
        target_grouping_label (str): The specific subclass label to filter for
            (e.g., "022 L5 ET CTX Glut").
        output_filename (str): The name of the .h5ad file to save the extracted data to.
        data_directory (str, optional): The base directory where the .h5ad files are located.
            If None, abc_atlas_cache.get_data_path will be used. Defaults to None.
        limit_matrices (int, optional): If provided, limits the number of feature matrices
            to process. Useful for testing. Defaults to None.
        additional_metadata_cols (list, optional): A list of column names from `cell_metadata`
            to be added to the `.obs` DataFrame of the output AnnData object.
            Defaults to None (no additional columns).

    Returns:
        str: The path to the newly created .h5ad file.
    """

    extracted_adata_list = []
    processed_count = 0
    total_start_time = time.process_time()

    print(f"Starting extraction for grouping: '{target_grouping_label}'...")

    for mat_index in feature_matrices_info.index:
        ds = mat_index[0] # dataset_id
        mp = mat_index[1] # matrix_path (e.g., "WMB-10x_nuclei-001/log2")

        print(f"Processing matrix: {mp}")

        # Construct the full path to the h5ad file
        if data_directory:
            file_path = os.path.join(data_directory, mp, 'log2') # Assuming 'log2' is part of the filename
        else:
            # Use abc_atlas_cache if data_directory is not provided
            file_path = abc_cache.get_data_path(directory=ds, file_name=os.path.join(mp, 'log2'))

        # Filter cell_metadata for cells belonging to the current matrix file AND the target subclass
        # This is crucial because 'cell_metadata' contains info for all cells,
        # but 'ad' only contains data for cells within the current matrix file.
        cells_in_current_matrix = cell_metadata[cell_metadata['feature_matrix_label'] == mp]
        target_cells_in_matrix = cells_in_current_matrix[
            cells_in_current_matrix[target_grouping_level] == target_grouping_label
        ]

        # Get the cell IDs (index) that match both criteria
        cell_ids_to_extract = target_cells_in_matrix.index

        if not cell_ids_to_extract.empty:
            start_time = time.process_time()
            try:
                # Read the AnnData object in backed mode for efficiency
                current_adata = ad.read_h5ad(file_path, backed='r')

                # Subset the AnnData object to include only the target cells and ALL genes
                # Use .to_memory() to load the subset into RAM before appending
                subset_adata = current_adata[cell_ids_to_extract, :].to_memory()

                # Merge additional metadata columns into subset_adata.obs
                if additional_metadata_cols:
                    for col in additional_metadata_cols:
                        if col in target_cells_in_matrix.columns:
                            # Ensure indices align for merging
                            subset_adata.obs[col] = target_cells_in_matrix.loc[subset_adata.obs_names, col]
                        else:
                            print(f" - Warning: Column '{col}' not found in cell_metadata. Skipping.")

                extracted_adata_list.append(subset_adata)
                print(f" - Extracted {subset_adata.n_obs} cells from {mp}. Time taken: {time.process_time() - start_time:.2f} seconds")

            except FileNotFoundError:
                print(f" - Warning: File not found for {mp} at {file_path}. Skipping.")
            except Exception as e:
                print(f" - Error processing {mp}: {e}. Skipping.")
            finally:
                # Ensure the file handle is closed if opened in backed mode
                if 'current_adata' in locals() and current_adata.file is not None:
                    current_adata.file.close()
                del current_adata # Free up memory
        else:
            print(f" - No cells found for grouping '{target_grouping_label}' in matrix {mp}. Skipping.")

        processed_count += 1
        if limit_matrices is not None and processed_count >= limit_matrices:
            print(f"Stopping after processing {limit_matrices} matrices as requested.")
            break

    print("\nConcatenating extracted AnnData objects...")
    if extracted_adata_list:
        # Concatenate all collected AnnData objects
        # 'join='outer'' and 'merge='unique'' ensure all genes and unique obs columns are kept
        final_adata = ad.concat(extracted_adata_list, axis=0, join='outer', merge='unique')
        print(f"Total cells extracted: {final_adata.n_obs}")
        print(f"Total genes retained: {final_adata.n_vars}")

        # Save the final AnnData object to a new h5ad file
        final_adata_path = output_filename
        final_adata.write(final_adata_path)
        print(f"Extracted data saved to: {final_adata_path}")
    else:
        print("No cells were extracted for the specified grouping level.")
        final_adata_path = None

    print(f"Total time taken for extraction: {time.process_time() - total_start_time:.2f} seconds")
    return final_adata_path

Now define the target taxonomic level with `target_grouping_level` and the label of the group within that level with `target_grouping_label`.

If you want to test the function on a subset of the input matrices, uncomment the `limit_matrices` line and it will break after the specified number of matrices. 

In [ ]:
# Define your target subclass and output filename
target_subclass = "022 L5 ET CTX Glut"
output_h5ad_file = "l5_et_ctx_cells_all_genes.h5ad"

# Call the function
extracted_file = extract_subclass_to_h5ad(
    cell_metadata=cell_metadata,
    feature_matrices_info=matrices,
    target_grouping_level="subclass",
    target_grouping_label=target_subclass,
    output_filename=output_h5ad_file,
    additional_metadata_cols=[ 'cell_barcode', # this can be any of the columns found in the `cell_metadata`
                                 'feature_matrix_label',
                                 'entity',
                                 'region_of_interest_acronym',
                                 'donor_sex',
                                 'dataset_label',
                                 'x',
                                 'y',
                                 'cluster_alias',
                                 'neurotransmitter',
                                 'class',
                                 'subclass',
                                 'supertype',
                                 'cluster',
                                 'neurotransmitter_color',
                                 'class_color',
                                 'subclass_color',
                                 'supertype_color',
                                 'cluster_color',
                                 'region_of_interest_order',
                                 'region_of_interest_color'], # Add columns you want to merge
    # data_directory="/path/to/your/abc_atlas_data" # Uncomment if not using abc_atlas_cache for file paths
    #limit_matrices=5 # Uncomment for quick testing with a few matrices
)

In [ ]:
# You can then load and work with the new h5ad file:
if extracted_file:
    l5_et_adata = ad.read_h5ad(extracted_file)
    print(l5_et_adata)

In [ ]:
# save the new h5ad
l5_et_adata.write_h5ad('../results/l5_et_adata.h5ad')